# Llama Stack Inference Guide

This document provides instructions on how to use Llama Stack's `chat_completion` function for generating text using the `Llama3.2-11B-Vision-Instruct` model. 

Before you begin, please ensure Llama Stack is installed and set up by following the [Getting Started Guide](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).


### Table of Contents
1. [Quickstart](#quickstart)
2. [Building Effective Prompts](#building-effective-prompts)
3. [Conversation Loop](#conversation-loop)
4. [Conversation History](#conversation-history)
5. [Streaming Responses](#streaming-responses)


## Quickstart

This section walks through each step to set up and make a simple text generation request.



### 0. Configuration
Set up your connection parameters:

In [1]:
HOST = "localhost"  # Replace with your host
PORT = 5000       # Replace with your port

### 1. Set Up the Client

Begin by importing the necessary components from Llama Stack’s client library:

In [2]:
from llama_stack_client import LlamaStackClient
from llama_stack_client.types import SystemMessage, UserMessage

client = LlamaStackClient(base_url=f'http://{HOST}:{PORT}')

### 2. Create a Chat Completion Request

Use the `chat_completion` function to define the conversation context. Each message you include should have a specific role and content:

In [3]:
response = client.inference.chat_completion(
    messages=[
        SystemMessage(content='You are a friendly assistant.', role='system'),
        UserMessage(content='Write a two-sentence poem about llama.', role='user')
    ],
    model='Llama3.2-11B-Vision-Instruct',
)

print(response.completion_message.content)

A gentle llama roams the land,
With soft fur and a gentle hand.


## Building Effective Prompts

Effective prompt creation (often called 'prompt engineering') is essential for quality responses. Here are best practices for structuring your prompts to get the most out of the Llama Stack model:

1. **System Messages**: Use `SystemMessage` to set the model's behavior. This is similar to providing top-level instructions for tone, format, or specific behavior.
   - **Example**: `SystemMessage(content='You are a friendly assistant that explains complex topics simply.')`
2. **User Messages**: Define the task or question you want to ask the model with a `UserMessage`. The clearer and more direct you are, the better the response.
   - **Example**: `UserMessage(content='Explain recursion in programming in simple terms.')`

### Sample Prompt

In [4]:
response = client.inference.chat_completion(
    messages=[
        SystemMessage(content='You are shakespeare.', role='system'),
        UserMessage(content='Write a two-sentence poem about llama.', role='user')
    ],
    model='Llama3.2-11B-Vision-Instruct',
)

print(response.completion_message.content)

O, fairest llama, with thy softest fleece,
Thy gentle eyes, like sapphires, in serenity do cease.


## Conversation Loop

To create a continuous conversation loop, where users can input multiple messages in a session, use the following structure. This example runs an asynchronous loop, ending when the user types 'exit,' 'quit,' or 'bye.'

In [5]:
import asyncio
from llama_stack_client import LlamaStackClient
from llama_stack_client.types import UserMessage
from termcolor import cprint

client = LlamaStackClient(base_url=f'http://{HOST}:{PORT}')

async def chat_loop():
    while True:
        user_input = input('User> ')
        if user_input.lower() in ['exit', 'quit', 'bye']:
            cprint('Ending conversation. Goodbye!', 'yellow')
            break

        message = UserMessage(content=user_input, role='user')
        response = client.inference.chat_completion(
            messages=[message],
            model='Llama3.2-11B-Vision-Instruct',
        )
        cprint(f'> Response: {response.completion_message.content}', 'cyan')

# Run the chat loop in a Jupyter Notebook cell using `await`
await chat_loop()
# To run it in a python file, use this line instead
# asyncio.run(chat_loop())

User>  Write me a 3 sentence poem about alpaca


> Response: Softly grazing, gentle soul,
Alpaca's fleece, a treasure whole,
In Andean fields, they softly roll.


User>  exit


Ending conversation. Goodbye!


## Conversation History

Maintaining a conversation history allows the model to retain context from previous interactions. Use a list to accumulate messages, enabling continuity throughout the chat session.

In [6]:
async def chat_loop():
    conversation_history = []
    while True:
        user_input = input('User> ')
        if user_input.lower() in ['exit', 'quit', 'bye']:
            cprint('Ending conversation. Goodbye!', 'yellow')
            break

        user_message = UserMessage(content=user_input, role='user')
        conversation_history.append(user_message)

        response = client.inference.chat_completion(
            messages=conversation_history,
            model='Llama3.2-11B-Vision-Instruct',
        )
        cprint(f'> Response: {response.completion_message.content}', 'cyan')

        assistant_message = UserMessage(content=response.completion_message.content, role='user')
        conversation_history.append(assistant_message)

# Use `await` in the Jupyter Notebook cell to call the function
await chat_loop()
# To run it in a python file, use this line instead
# asyncio.run(chat_loop())

User>  what is 1+1


> Response: 1 + 1 = 2


User>  what is llama + alpaca


> Response: That's a creative and imaginative question. However, since llamas and alpacas are animals, not numbers, we can't perform a mathematical operation on them.

But if we were to interpret this as a creative or humorous question, we could say that the result of "llama + alpaca" is a fun and fuzzy bundle of South American camelid cuteness!


User>  what was the first question


> Response: The first question was "what is 1+1"


User>  exit


Ending conversation. Goodbye!


## Streaming Responses

Llama Stack offers a `stream` parameter in the `chat_completion` function, which allows partial responses to be returned progressively as they are generated. This can enhance user experience by providing immediate feedback without waiting for the entire response to be processed.

### Example: Streaming Responses

In [7]:
import asyncio
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.inference.event_logger import EventLogger
from llama_stack_client.types import UserMessage
from termcolor import cprint

async def run_main(stream: bool = True):
    client = LlamaStackClient(base_url=f'http://{HOST}:{PORT}')

    message = UserMessage(
        content='Write me a 3 sentence poem about llama', role='user'
    )
    cprint(f'User> {message.content}', 'green')

    response = client.inference.chat_completion(
        messages=[message],
        model='Llama3.2-11B-Vision-Instruct',
        stream=stream,
    )

    if not stream:
        cprint(f'> Response: {response.completion_message.content}', 'cyan')
    else:
        async for log in EventLogger().log(response):
            log.print()

# In a Jupyter Notebook cell, use `await` to call the function
await run_main()
# To run it in a python file, use this line instead
# asyncio.run(chat_loop())

User> Write me a 3 sentence poem about llama
Assistant> Softly padded feet on the ground,
A gentle llama's peaceful sound,
Furry coat and calm, serene eyes all around.
